In [1]:
# Uninstall packages to make sure most up to date are imported below
import sys
!{sys.executable} -m pip uninstall -y lusid-notifications-sdk-preview
!{sys.executable} -m pip uninstall -y lusid-workflows-sdk-generated-preview
!{sys.executable} -m pip uninstall -y finbourne-sdk-utilities

Found existing installation: lusid-notifications-sdk-preview 0.1.260
Uninstalling lusid-notifications-sdk-preview-0.1.260:
  Successfully uninstalled lusid-notifications-sdk-preview-0.1.260


Found existing installation: finbourne-sdk-utilities 0.0.7
Uninstalling finbourne-sdk-utilities-0.0.7:
  Successfully uninstalled finbourne-sdk-utilities-0.0.7


## Notifications Test

This notebook runs through a use case for the notification endpoints. The use case is that, following a data load into LUSID (for example daily feed of Transactions) :

* A subscription is created to report the upload event
* An email notification is added to the subscription to report the event to a set of interested parties
* A webhook notification is added to the subscription to continue processing in a downstream system (this is currently disabled)
* The event is manually triggered

In [2]:
# Import general purpose packages
import os
import json
from datetime import datetime, timedelta
import pytz

# Import lusid specific packages
import lusid
import lusid.models as models
from lusid.utilities import ApiClientFactory as lusid_factory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.seed_sample_data import seed_data
from lusidtools.cocoon.utilities import create_scope_id

# Import notifications and workflows sdk preview packages
import sys
!{sys.executable} -m pip install lusid-notifications-sdk-preview
!{sys.executable} -m pip install lusid-workflows-sdk-preview
!{sys.executable} -m pip install finbourne-sdk-utilities
import lusid_notifications as ln
import lusid_workflows as lw
from fbnsdkutilities import ApiClientFactory
# Import data wrangling packages
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)

# Authenticate our user and create our API client
from lusidtools.cocoon import load_from_data_frame
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
lusid_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

lusid_api_url = lusid_factory.api_client.configuration.host
notifications_url = lusid_api_url[: lusid_api_url.rfind("/") + 1] + "notifications"
workflows_url = lusid_api_url[: lusid_api_url.rfind("/") + 1] + "workflows"

notifications_factory = ApiClientFactory(
    sdk=ln,
    token=RefreshingToken(),
    api_url=notifications_url
)

workflows_factory = ApiClientFactory(
    sdk=lw,
    token=RefreshingToken(),
    api_url=workflows_url
)

# Setup the apis we'll use in this notebook:
subscriptions_api = notifications_factory.build(ln.api.SubscriptionsApi)
notifications_api = notifications_factory.build(ln.api.NotificationsApi)
events_api = notifications_factory.build(ln.api.EventsApi)


print ('LUSID Environment Initialised')
print ('LUSID SDK Version: ', lusid_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

  Using cached lusid_notifications_sdk_preview-0.1.260-py3-none-any.whl (93 kB)


You should consider upgrading via the 'C:\Python38\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Python38\python.exe -m pip install --upgrade pip' command.


  Using cached finbourne_sdk_utilities-0.0.7-py3-none-any.whl (16 kB)


You should consider upgrading via the 'C:\Python38\python.exe -m pip install --upgrade pip' command.


ValueError: The fields ['api_url', 'password', 'username', 'client_id', 'client_secret', 'token_url'] on the ApiConfiguration are set to None, please ensure that you have provided them directly, via a secrets file or environment variables

### Create event subscriptions

These cells create subscriptions to the events that will be used in the notebook. From these subscriptions notifications can be set up to alert various users or automically kick off follow on actions.

In [ ]:
# create subscriptions to our manual events used in this notebook

def create_event_subscriptions(subscription_scope, subscription_code, display_name, event_filter):
    event_subscription = ln.CreateSubscription(
                id= ln.ResourceId(
                    scope=subscription_scope,
                    code=subscription_code
                ),
                display_name=display_name,
                description="Subscription to a manual event",
                status="Active",
                matching_pattern= ln.MatchingPattern(
                    event_type="Manual",
                    filter="Message eq '"+event_filter+"'"
                )
    )
    
    # create subscription definition
    try:
        create_subscription_event = subscriptions_api.create_subscription(
            create_subscription=event_subscription
        )

    except lusid.ApiException as e:
        print(json.loads(e.body))

    return event_subscription

In [ ]:
subscription_scope = 'subscriptionscope'
subscription_code = 'TransactionsLoaded'

In [ ]:
# Setup the subscription details
event_filter ='TransactionsLoaded'
display_name='TransactionsLoaded'
event_subscription = create_event_subscriptions(subscription_scope, subscription_code, display_name, event_filter)

### Create email notifications

This cell sets up email notifications for the events that have been subscribed to above. This means the user will recieve an email for each of the events triggered throughout this notebook.

In [ ]:
# create email notifications for the subscription
create_email_notification= ln.CreateEmailNotification(
            description="Email Event Notification",
            subject="Email Event Notification",
            plain_text_body="Event with message and details",
            email_address_to=["lorenz.declercq@finbourne.com"]
            )

# Create a Webhook notification for the subscription
#
# create_api_request_notification=ln.CreateWebhookNotification(
#     description="Makes an Issue custom entity",
#     http_method="Post",
#     path_and_query="api/api/customentities/~issue",
#     content = "example request")
    
try:
    create_email_notifications_response = notifications_api.create_email_notification(subscription_scope, subscription_code, create_email_notification
    )
#   create_api_notification_response = notifications_api.create_api_request_notification(subscription_scope, subscription_code, create_api_request_notification)
    notifications = notifications_api.list_notifications(subscription_scope, subscription_code)
    print(notifications)

except lusid.ApiException as e:
    print(e)


In [ ]:
# create manual event for a new issue so that it triggers the email notification
body= dict(
    EventType = "Manual",
    Id = "abcdefghijk12345",
    Message = "TransactionsLoaded",
            subject="Email Event Notification",    
    Details ="Test details",
    EventTime ="2021-08-27T17:39:02.9427036+01:00"
)

try:
    create_event_response = events_api.create_event(body)
    print(create_event_response)
except lusid.ApiException as e:
    print(json.loads(e.body))

In [ ]:
# Remove the subscription
subscriptions_api.delete_subscription(scope=subscription_scope, code=subscription_code)